In [ ]:
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, transpile
from qiskit_aer import AerSimulator

from LogicalQ.Logical import LogicalCircuit

from LogicalQ.NoiseModel import (
    construct_noise_model,
    construct_noise_model_QuantinuumH1_1,
    construct_noise_model_QuantinuumH2_1,
    construct_noise_model_QuantinuumH2_2,
)

from LogicalQ.Benchmarks import (
    n_qubit_ghz_generation,
    generate_quantum_teleportation_circuit,
    mirror_benchmarking,
    randomized_benchmarking,
    quantum_volume,
)

from LogicalQ.Experiments import (
    execute_circuits,
    circuit_scaling_experiment,
    noise_scaling_experiment,
    qec_cycle_efficiency_experiment,
    qec_cycle_noise_scaling_experiment,
)

from LogicalQ.Analysis import (
    circuit_scaling_bar3d,
    noise_model_scaling_bar,
    qec_cycle_efficiency_bar,
)

In [ ]:
# We use the Steane code to demonstrate the library
steane_stabilizer_tableau = [
  "XXXXIII",
  "IXXIXXI",
  "IIXXIXX",
  "ZZZZIII",
  "IZZIZZI",
  "IIZZIZZ",
]

# Benchmarks

In [ ]:
# Build one instance of each benchmark circuit so we can re-use them

bench_ghz2 = n_qubit_ghz_generation(n_qubits=2, barriers=False)
bench_ghz3 = n_qubit_ghz_generation(n_qubits=3, barriers=False)

# Teleportation
bench_tele3 = generate_quantum_teleportation_circuit(statevector=[1, 0], n_qubits=3, barriers=False)

# Mirror benchmarking
bench_mb2 = mirror_benchmarking(n_qubits=2, circuit_length=4)

# Randomized benchmarking (1‑qubit short demo)
bench_rb1 = randomized_benchmarking(n_qubits=[0], circuit_lengths=[2, 8], num_samples=2, seed=11)

# Quantum volume (2‑qubit, 2 trials)
bench_qv2 = quantum_volume(n_qubits=2, trials=2, seed=7)

print("Benchmarks built")

# Noise Models

In [ ]:
BASIS_GATES = ["x","y","z","h","s","t","rx","ry","rz","cx","cy","cz","ch","measure"]

nm_generic_2 = construct_noise_model(BASIS_GATES, n_qubits=2)
nm_generic_3 = construct_noise_model(BASIS_GATES, n_qubits=3)

nm_H1_1 = construct_noise_model_QuantinuumH1_1(n_qubits=2)
nm_H2_1 = construct_noise_model_QuantinuumH2_1(n_qubits=2)
nm_H2_2 = construct_noise_model_QuantinuumH2_2(n_qubits=2)

print("Noise models built")

# Circuit Scaling

In [ ]:
# Construct circuit factories as circuit inputs
physical_circuit_factory = n_qubit_ghz_generation

def log_n_qubit_ghz_generation(n_qubits=3, circuit_length=None, barriers=False):
    pqc = n_qubit_ghz_generation(n_qubits, circuit_length, barriers=barriers)
    lqc = LogicalCircuit.from_physical_circuit(pqc, label=(7,1,3), stabilizer_tableau=steane_stabilizer_tableau)
    lqc.measure_all(with_error_correction=False)
    return lqc

logical_circuit_factory = log_n_qubit_ghz_generation

# Construct noise model factories as noise model inputs
noise_model_factory = construct_noise_model_QuantinuumH1_1

## Serial

In [ ]:
phys_data_serial = circuit_scaling_experiment(
    physical_circuit_factory,
    noise_model_factory,
    min_n_qubits=1,
    max_n_qubits=2,
    min_circuit_length=1,
    max_circuit_length=64,
    shots=128,
    with_mp=False
)

In [ ]:
log_data_serial = circuit_scaling_experiment(
    logical_circuit_factory,
    noise_model_factory,
    min_n_qubits=1,
    max_n_qubits=2,
    min_circuit_length=1,
    max_circuit_length=1,
    shots=128,
    with_mp=False
)

In [ ]:
circuit_scaling_bar3d(phys_data_serial, title="Physical circuit scaling (serial)", show=True)

In [ ]:
circuit_scaling_bar3d(log_data_serial, title="Logical circuit scaling (serial)", show=True)

## With MP

In [ ]:
phys_data_mp = circuit_scaling_experiment(
    physical_circuit_factory,
    noise_model_factory,
    min_n_qubits=1,
    max_n_qubits=2,
    min_circuit_length=1,
    max_circuit_length=64,
    shots=128,
    with_mp=True
)

In [ ]:
log_data_mp = circuit_scaling_experiment(
    logical_circuit_factory,
    noise_model_factory,
    min_n_qubits=1,
    max_n_qubits=2,
    min_circuit_length=1,
    max_circuit_length=1,
    shots=128,
    with_mp=True
)

In [ ]:
circuit_scaling_bar3d(phys_data_mp, title="Physical circuit scaling (mp)", show=True)

In [ ]:
circuit_scaling_bar3d(log_data_mp, title="Logical circuit scaling (mp)", show=True)

# Noise Model Scaling

In [ ]:
# Sweep 3x3x3 = 27 combinations of noise values.
basis_gates = ["x","y","z","h","s","t","rx","ry","rz","cx","cy","cz","ch","measure"]
lqc = log_n_qubit_ghz_generation(n_qubits=3)
phys3 = n_qubit_ghz_generation(n_qubits=3, barriers=False)

hardware_spec_phys = {
    "device_info": {
        "n_qubits": phys3.num_qubits,
        "coupling_map": None,
        "basis_gates": BASIS_GATES
    },
    "noise_params": {
        "all_qubit": {
            "amplitude_1q": 0.0,
            "depolarizing_cx": 0.0,
            "readout_0|1": 0.0
        }
    }
}

hardware_spec_logical = {
    "device_info": {
        "n_qubits": lqc.num_qubits,
        "coupling_map": None,
        "basis_gates": BASIS_GATES
    },
    "noise_params": {
        "all_qubit": {
            "amplitude_1q": 0.0,
            "depolarizing_cx": 0.0,
            "readout_0|1": 0.0
        }
    }
}

noise_param_scan = {
    "all_qubit.amplitude_1q": [0.0, 2e-4, 1e-3], # 1q amplitude
    "all_qubit.depolarizing_cx": [0.0, 5e-3, 2e-2], # 2q depol
    "all_qubit.readout_0|1": [0.0, 5e-3, 5e-2] # readout 0->1
}

base_noise_model = construct_noise_model_QuantinuumH1_1()

## Logical circuit

In [ ]:
noise_data_logical_all_true = noise_scaling_experiment(
    circuit_input=lqc,
    hardware_spec=hardware_spec_logical,
    noise_param_scan=noise_param_scan,
    compute_exact=True,
    shots=0,
    with_mp=True
)

noise_model_scaling_bar(noise_data_logical_all_true, scan_keys=error_scan_keys, separate_plots=True)

In [ ]:
noise_data_logical_all_false = noise_scaling_experiment(
    circuit_input=lqc,
    hardware_spec=hardware_spec_logical,
    noise_param_scan=noise_param_scan,
    compute_exact=False,
    shots=256,
    with_mp=False
)

noise_model_scaling_bar(noise_data_logical_all_false, scan_keys=error_scan_keys, separate_plots=False)

## Physical circuit

In [ ]:
noise_data_phys_all_true = noise_scaling_experiment(
    circuit_input=phys3,
    hardware_spec=hardware_spec_phys,
    noise_param_scan=noise_param_scan,
    compute_exact=True,
    shots=0,
    with_mp=True
)


noise_model_scaling_bar(noise_data_phys_exact_mp, scan_keys=error_scan_keys, separate_plots=True)

In [ ]:
noise_data_phys_all_false = noise_scaling_experiment(
    circuit_input=phys3,
    hardware_spec=hardware_spec_phys,
    noise_param_scan=noise_param_scan,
    compute_exact=False,
    shots=256,
    with_mp=False
)

noise_model_scaling_bar(noise_data_phys_mp, scan_keys=error_scan_keys, separate_plots=False)

# QEC Cycle Efficiency

In [ ]:
# Inject 0–4 cycles of Steane QEC and see how fidelity/num_cycle behaves.
# After n cycles, how much logical fidelity have we recovered per cycle?
config_scan_keys = ["cycles"]
config_scan_val_lists = [list(range(5))] 

# Build logical GHZ circuit (3 logical qubits here for demo)
lqc_single = log_n_qubit_ghz_generation(n_qubits=3)

# Concrete noise model (base Quantinuum H1-1)
base_noise_model = construct_noise_model_QuantinuumH1_1()

## Serial

In [ ]:
qec_eff_data_serial = qec_cycle_efficiency_experiment(
    circuit_inputs = [lqc_single],
    noise_model_input = base_noise_model,
    config_scan_keys = config_scan_keys,
    config_scan_val_lists = config_scan_val_lists,
    method = "density_matrix",
    shots = 256,
    with_mp=False
)

In [ ]:
qec_cycle_efficiency_bar(qec_eff_data_serial)

## With MP

In [ ]:
qec_eff_data_mp = qec_cycle_efficiency_experiment(
    circuit_inputs = [lqc_single],
    noise_model_input = base_noise_model,
    config_scan_keys = config_scan_keys,
    config_scan_val_lists = config_scan_val_lists,
    method = "density_matrix",
    shots = 256,
    with_mp=True
)

In [ ]:
qec_cycle_efficiency_bar(qec_eff_data_mp)

# QEC Cycle Noise Scaling

In [ ]:
hardware_spec_qec = {
    "device_info": {
        "n_qubits": lqc_single.num_qubits,
        "coupling_map": None,
        "basis_gates": BASIS_GATES
    },
    "noise_params": {
        "all_qubit": {
            "amplitude_1q":    0.0,
            "depolarizing_cx": 0.0,
            "readout_0|1":     0.0
        }
    }
}


noise_param_scan_qec = {
    "all_qubit.amplitude_1q": [0.0, 1e-2]
}

In [ ]:
qec_noise_scaling_serial = qec_cycle_noise_scaling_experiment(
    circuit_input=lqc_single,
    hardware_spec = hardware_spec_qec,
    noise_param_scan = noise_param_scan_qec,
    logical_kwargs=None,
    save_filename=None,
    with_mp=False
)

In [ ]:
qec_noise_scaling_mp = qec_cycle_noise_scaling_experiment(
    circuit_input=lqc_single,
    hardware_spec = hardware_spec_qec,
    noise_param_scan = noise_param_scan_qec,
    logical_kwargs=None,
    save_filename=None,
    with_mp=True
)